Importing Required Libraries

In [ ]:
import string
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install scispacy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip3 install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Computing For Medicine Project/mtsamples.csv")

In [ ]:
data.head(10)

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."
5,5,Morbid obesity. Laparoscopic antecolic anteg...,Bariatrics,Laparoscopic Gastric Bypass,"PREOPERATIVE DIAGNOSIS: , Morbid obesity.,POST...","bariatrics, gastric bypass, eea anastomosis, r..."
6,6,"Liposuction of the supraumbilical abdomen, re...",Bariatrics,Liposuction,"PREOPERATIVE DIAGNOSES:,1. Deformity, right b...","bariatrics, breast reconstruction, excess, lma..."
7,7,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 3,"2-D ECHOCARDIOGRAM,Multiple views of the heart...","cardiovascular / pulmonary, 2-d echocardiogram..."
8,8,Suction-assisted lipectomy - lipodystrophy of...,Bariatrics,Lipectomy - Abdomen/Thighs,"PREOPERATIVE DIAGNOSIS: , Lipodystrophy of the...","bariatrics, lipodystrophy, abd pads, suction-a..."
9,9,Echocardiogram and Doppler,Cardiovascular / Pulmonary,2-D Echocardiogram - 4,"DESCRIPTION:,1. Normal cardiac chambers size....","cardiovascular / pulmonary, ejection fraction,..."


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         4999 non-null   int64 
 1   description        4999 non-null   object
 2   medical_specialty  4999 non-null   object
 3   sample_name        4999 non-null   object
 4   transcription      4966 non-null   object
 5   keywords           3931 non-null   object
dtypes: int64(1), object(5)
memory usage: 234.5+ KB


In [ ]:
data.shape

(4999, 6)

In [ ]:
data.columns

Index(['Unnamed: 0', 'description', 'medical_specialty', 'sample_name',
       'transcription', 'keywords'],
      dtype='object')

In [ ]:
data = data.drop('Unnamed: 0', axis = 1)

In [ ]:
data = data.dropna(subset=['transcription'])
data.shape

(4966, 5)

In [ ]:
category_count_series = data['medical_specialty'].value_counts()
category_count_series

 Surgery                          1088
 Consult - History and Phy.        516
 Cardiovascular / Pulmonary        371
 Orthopedic                        355
 Radiology                         273
 General Medicine                  259
 Gastroenterology                  224
 Neurology                         223
 SOAP / Chart / Progress Notes     166
 Urology                           156
 Obstetrics / Gynecology           155
 Discharge Summary                 108
 ENT - Otolaryngology               96
 Neurosurgery                       94
 Hematology - Oncology              90
 Ophthalmology                      83
 Nephrology                         81
 Emergency Room Reports             75
 Pediatrics - Neonatal              70
 Pain Management                    61
 Psychiatry / Psychology            53
 Office Notes                       50
 Podiatry                           47
 Dermatology                        29
 Cosmetic / Plastic Surgery         27
 Dentistry               

In [ ]:
sum(category_count_series.values.tolist())

4966

In [ ]:
def get_relevant_columns(category_count_series , THRESHOLD):
  # returns only the columns with unique count > THRESHOLD
  res = []
  for index_val in category_count_series.iteritems():
    if index_val[1] >= THRESHOLD:
      res.append(index_val[0])
    else:
      break
  return res

In [ ]:
filtered_columns = get_relevant_columns(category_count_series , 100)
filtered_columns

[' Surgery',
 ' Consult - History and Phy.',
 ' Cardiovascular / Pulmonary',
 ' Orthopedic',
 ' Radiology',
 ' General Medicine',
 ' Gastroenterology',
 ' Neurology',
 ' SOAP / Chart / Progress Notes',
 ' Urology',
 ' Obstetrics / Gynecology',
 ' Discharge Summary']

In [ ]:
filtered_data = data[data.medical_specialty.isin(filtered_columns) == True]
filtered_data

,description,medical_specialty,sample_name,transcription,keywords
3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."
7,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 3,"2-D ECHOCARDIOGRAM,Multiple views of the heart...","cardiovascular / pulmonary, 2-d echocardiogram..."
9,Echocardiogram and Doppler,Cardiovascular / Pulmonary,2-D Echocardiogram - 4,"DESCRIPTION:,1. Normal cardiac chambers size....","cardiovascular / pulmonary, ejection fraction,..."
11,"Normal left ventricle, moderate biatrial enla...",Cardiovascular / Pulmonary,2-D Doppler,"2-D STUDY,1. Mild aortic stenosis, widely calc...","cardiovascular / pulmonary, 2-d study, doppler..."
...,...,...,...,...,...
4972,Adenosine with nuclear scan as the patient un...,Cardiovascular / Pulmonary,Adenosine Nuclear Scan,"INDICATION: , Chest pain.,TYPE OF TEST: , Aden...","cardiovascular / pulmonary, adenosine nuclear ..."
4973,Patient presents with a chief complaint of ch...,Cardiovascular / Pulmonary,Acute Inferior Myocardial Infarction,"CHIEF COMPLAINT: , Chest pain.,HISTORY OF PRES...",NaN
4974,Patient had a recurrent left arm pain after h...,Cardiovascular / Pulmonary,Angina - Consult,"HISTORY OF PRESENT ILLNESS: , The patient is a...",NaN
4975,Patient had a markedly abnormal stress test w...,Cardiovascular / Pulmonary,Abnormal Stress Test,"HISTORY OF PRESENT ILLNESS: , Mr. ABC is a 60-...","cardiovascular / pulmonary, standard bruce, ni..."


In [ ]:
filtered_data['medical_specialty'].value_counts()

 Surgery                          1088
 Consult - History and Phy.        516
 Cardiovascular / Pulmonary        371
 Orthopedic                        355
 Radiology                         273
 General Medicine                  259
 Gastroenterology                  224
 Neurology                         223
 SOAP / Chart / Progress Notes     166
 Urology                           156
 Obstetrics / Gynecology           155
 Discharge Summary                 108
Name: medical_specialty, dtype: int64

In [ ]:
filtered_data = filtered_data[['transcription', 'medical_specialty']]
filtered_data = filtered_data.drop(filtered_data[filtered_data['transcription'].isna()].index)
filtered_data

,transcription,medical_specialty
3,"2-D M-MODE: , ,1. Left atrial enlargement wit...",Cardiovascular / Pulmonary
4,1. The left ventricular cavity size and wall ...,Cardiovascular / Pulmonary
7,"2-D ECHOCARDIOGRAM,Multiple views of the heart...",Cardiovascular / Pulmonary
9,"DESCRIPTION:,1. Normal cardiac chambers size....",Cardiovascular / Pulmonary
11,"2-D STUDY,1. Mild aortic stenosis, widely calc...",Cardiovascular / Pulmonary
...,...,...
4972,"INDICATION: , Chest pain.,TYPE OF TEST: , Aden...",Cardiovascular / Pulmonary
4973,"CHIEF COMPLAINT: , Chest pain.,HISTORY OF PRES...",Cardiovascular / Pulmonary
4974,"HISTORY OF PRESENT ILLNESS: , The patient is a...",Cardiovascular / Pulmonary
4975,"HISTORY OF PRESENT ILLNESS: , Mr. ABC is a 60-...",Cardiovascular / Pulmonary


In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords

def remove_punctuation(text):
  for char in string.punctuation:
    text = text.replace(char, '')
  return text

lemmatizer = WordNetLemmatizer()
def use_lemmatizer(text):
  ret_text = ""
  use_text = ""
  for char in text:
    if not char.isdigit():
      use_text += char
  
  words = word_tokenize(use_text)
  res_tokens = []
  for word in words:
    # ret_text += lemmatizer.lemmatize(word) + " "
    res_tokens.append(lemmatizer.lemmatize(word).lower())

  return res_tokens

def remove_stop_words(df):
  stop_words_ = stopwords.words('english')
  # df['transcription'] = df['transcription'].apply(stop_words_func)
  df['transcription'] = df['transcription'].apply(lambda x: ' '.join([word for word in x if word not in (stop_words_)]))
  

In [ ]:
!pip install "/content/drive/MyDrive/Computing For Medicine Project/en_ner_bionlp13cg_md-0.5.1.tar.gz"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./drive/MyDrive/Computing For Medicine Project/en_ner_bionlp13cg_md-0.5.1.tar.gz
DEPRECATION: Source distribution is being reinstalled despite an installed package having the same name and version as the installed package. pip 21.2 will remove support for this functionality. A possible replacement is use --force-reinstall. You can find discussion regarding this at https://github.com/pypa/pip/issues/8711.
  Created wheel for en-ner-bionlp13cg-md: filename=en_ner_bionlp13cg_md-0.5.1-py3-none-any.whl size=120241162 sha256=c1c008de028fd43729402a6b116cb1ea69d8c508158b2b0cce665d508e8e4583
  Stored in directory: /root/.cache/pip/wheels/08/3f/d3/90211068135178bebdd1dfbfce6d8776d43d25eb9df36e9105
Successfully built en-ner-bionlp13cg-md
  Attempting uninstall: en-ner-bionlp13cg-md
    Found existing installation: en-ner-bionlp13cg-md 0.5.1
    Uninstalling en-ner-bionlp13cg-md-0.5.1:
   

In [ ]:
import spacy
import en_ner_bionlp13cg_md
bio_nlp = en_ner_bionlp13cg_md.load()

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
bio_nlp("The wall of cellular venticles").ents[1]

cellular venticles

In [ ]:
def process_bio_nlp(sentence):
  ret_text = ""
  doc = bio_nlp(sentence)
  for entity in doc.ents:
    ret_text += (entity.text + ' ')
  return ret_text

In [ ]:
process_bio_nlp(filtered_data['transcription'].iloc[2])

'heart vessels vessel vegetations venous atrium tricuspid inflow Pulmonary outflow Pulmonary venous left atrium interatrial septum Mitral inflow aorta aortic valve coronary arteries aortic arch aorta '

In [ ]:
# filtered_data['transcription'] = filtered_data['transcription'].apply(remove_punctuation)
filtered_data['transcription'] = filtered_data['transcription'].apply(process_bio_nlp)
filtered_data['transcription'] = filtered_data['transcription'].apply(use_lemmatizer)
remove_stop_words(filtered_data)
# filtered_data['transcription'] = filtered_data['transcription'].apply(use_bert_tokenizer)
filtered_data.head()

,transcription,medical_specialty
3,left atrial left lv systolic left ventricular ...,Cardiovascular / Pulmonary
4,left ventricular wall wall left ventricular le...,Cardiovascular / Pulmonary
7,heart vessel vessel vegetation venous atrium t...,Cardiovascular / Pulmonary
9,cardiac left ventricular lv systolic aortic va...,Cardiovascular / Pulmonary
11,"ventricular tricuspid mitral valves. , ventric...",Cardiovascular / Pulmonary


In [ ]:
print(filtered_data['transcription'].iloc[2656])

patient left tm amoxil ear amoxil vaginal noncontributory. , physical examination : ,vital nasal whitish extraocular muscle edema. , laboratory cbc rsv oral oxygen


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1,1), max_df=0.75, use_idf=True, smooth_idf=True, max_features=800)
tfIdfMat = vectorizer.fit_transform(filtered_data['transcription'].tolist())

In [ ]:
y_labels = filtered_data['medical_specialty'].tolist()

In [ ]:
tfIdfMat.shape

(3894, 800)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
tfIdfMat_pca = pca.fit_transform(tfIdfMat.toarray())

In [ ]:
def get_split_data(df , df_labels , test_size_= 0.2):
  X_train, X_test, y_train, y_test = train_test_split(df , df_labels , test_size=test_size_ , random_state = 42)
  return X_train , X_test , y_train , y_test

In [ ]:
X_train, X_test , y_train , y_test = get_split_data(tfIdfMat_pca, y_labels)

In [ ]:
X_train.shape

(3115, 526)

In [ ]:
all_possible_labels = filtered_data['medical_specialty'].unique().tolist()
label_index = [i for i in range(len(all_possible_labels))]
label_to_num = dict(zip(all_possible_labels , label_index))
# label_to_num
for i in range(len(y_train)):
  y_train[i] = label_to_num[y_train[i]]

for i in range(len(y_test)):
  y_test[i] = label_to_num[y_test[i]]


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
best_knn = KNeighborsClassifier(n_neighbors = 26)
best_knn.fit(X_train , y_train)
best_knn_out = best_knn.predict(X_test)
report = metrics.accuracy_score(y_test , best_knn_out)
print(report)

0.27086007702182285


In [ ]:
from sklearn.svm import SVC
best_svc = SVC(C=100 , gamma = 0.001 , kernel = 'rbf')
best_svc.fit(X_train , y_train)
svc_pred_out = best_svc.predict(X_test)
report = metrics.accuracy_score(y_test , svc_pred_out)
print(report)

0.4030808729139923


In [ ]:
from sklearn.linear_model import LogisticRegression  
clf = LogisticRegression(penalty= 'elasticnet', solver= 'saga', l1_ratio=0.5, random_state=1).fit(X_train, y_train)
y_test_pred= clf.predict(X_test)
report = metrics.accuracy_score(y_test , y_test_pred)
print(report)

0.4056482670089859
